In [1]:
import os
print(os.getcwd())
from theano.sandbox import cuda
import sys
sys.path.append('/app/dev/dhanley2/rnn')
import utils; reload(utils)
from utils import *
#from __future__ import division, print_function
from keras.layers import TimeDistributed, Activation
from numpy.random import choice

/app/dev/dhanley2/rnn


Using TensorFlow backend.


In [2]:
path = get_file('nietzsche.txt', origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt")
textn = open(path).read().lower()
print('corpus length:', len(textn), type(textn))


('corpus length:', 600901, <type 'str'>)


In [3]:
# path = get_file('dubliners.txt', origin="/app/dev/dhanley2/rnn/data")

with open('data/dubliners.txt', 'r') as myfile:
  text=myfile.read().replace('\n', ' ').replace('\r', ' ').lower()
# text = unicode(text, 'utf8')
print('corpus length:', len(text), type(text))

('corpus length:', 385838, <type 'str'>)


In [4]:
chars = sorted(list(set(text)))
vocab_size = len(chars)+1
print('total chars:', vocab_size)

('total chars:', 61)


In [5]:
chars.insert(0, "\0")
''.join(chars[1:-6])

' !&(),-.156789:;?abcdefghijklmnopqrstuvwxyz\x80\x93\x94\x98\x99\x9c\x9d\xa6\xa7\xa8\xa9'

In [6]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [7]:
idx = [char_indices[c] for c in text]

In [11]:
print(idx[:40])
''.join(indices_char[i] for i in idx[:40])

[60, 55, 56, 21, 38, 19, 29, 26, 31, 22, 35, 36, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 37, 25, 22, 1, 36, 26, 36, 37, 22, 35, 36, 1, 1, 1]


'\xef\xbb\xbfdubliners              the sisters   '

### Preprocess and create model

In [12]:
maxlen = 40
sentences = []
next_chars = []
for i in range(0, len(idx) - maxlen+1):
    sentences.append(idx[i: i + maxlen])
    next_chars.append(idx[i+1: i+maxlen+1])
print('nb sequences:', len(sentences))

('nb sequences:', 385799)


In [13]:
sentences = np.concatenate([[np.array(o)] for o in sentences[:-2]])
next_chars = np.concatenate([[np.array(o)] for o in next_chars[:-2]])

In [14]:
sentences.shape, next_chars.shape

((385797, 40), (385797, 40))

In [15]:
n_fac = 24

In [ ]:
model=Sequential([
        Embedding(vocab_size, n_fac, input_length=maxlen),
        LSTM(512, input_dim=n_fac,return_sequences=True, dropout_U=0.2, dropout_W=0.2,
             consume_less='gpu'),
        Dropout(0.2),
        LSTM(512, return_sequences=True, dropout_U=0.2, dropout_W=0.2,
             consume_less='gpu'),
        Dropout(0.2),
        TimeDistributed(Dense(vocab_size)),
        Activation('softmax')
    ])